In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn import metrics as m
from thundersvm import SVC as svmgpu
import calculateWeightUsingGa2 as aresult
import pandas as pd
import itertools
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import VotingClassifier

randomseed = 42
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

1. Read Dataset 
===

In [3]:
data = datasets.load_wine()
x = data.data
y = data.target

print(np.unique(y))

xtrain, xtest, ytrain_original, ytest_original = train_test_split(
    x, y, test_size=0.3, random_state=10
)

ytrain = ytrain_original.copy()
ytest = ytest_original.copy()

[0 1 2]


<IPython.core.display.Javascript object>

In [4]:
def swapcolumns(trainval, testval, coldindexval):
    trainval[trainval != coldindexval] = 5
    testval[testval != coldindexval] = 5

    trainval[trainval == coldindexval] = 0
    trainval[trainval == 5] = 1

    testval[testval == coldindexval] = 0
    testval[testval == 5] = 1

    return trainval, testval

<IPython.core.display.Javascript object>

In [5]:

original_scores=[]
trial1_scores=[]
trial2_scores=[]
trial3_scores=[]
trial4_scores=[]
trial5_scores=[]
trial6_scores=[]

from sklearn.model_selection import KFold 
kf = KFold(n_splits=10, random_state=randomseed, shuffle=True)
kf.get_n_splits(x)

print(kf)

data_cv=[]

for train_index, test_index in kf.split(x):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]    
    data_cv.append([[X_train, X_test],[y_train, y_test]])


for i in range(len(data_cv)):
    print('************************** ==> ',i)
    xtrain=data_cv[i][0][0]
    ytrain_original =data_cv[i][1][0] 
    xtest= data_cv[i][0][1]
    ytest_original= data_cv[i][1][1]
    
    
    # xtrain,xtest,ytrain_original,ytest_original=train_test_split(x,y,random_state=randomseed,test_size=0.3) 
    
    ytrain=ytrain_original.copy()
    ytest=ytest_original.copy() 
    
    # ytrain, ytest = swapcolumns(ytrain, ytest, 2)
    
    
    
    clf=[]
    acc=[]
    finalacc=[]
    ypredproba_all=[]
    ypredconfprob_all=[]
    
    rf=RandomForestClassifier(random_state=randomseed, n_estimators=100)
    rf.fit(xtrain,ytrain)
    print('original score',m.f1_score(ytest,rf.predict(xtest),average='weighted'))
    original_scores.append(m.f1_score(ytest,rf.predict(xtest),average='weighted'))
    
    #================================================= 
    
    # Class 0
    # ===========================
    ytrain = ytrain_original.copy()
    ytest = ytest_original.copy()
    ytrain,ytest= swapcolumns(ytrain,ytest,0)
    #=================================================
    
    rf=RandomForestClassifier(random_state=randomseed, n_estimators=100)
    rf.fit(xtrain,ytrain)
    rfpred=rf.predict(xtest)
    print(m.f1_score(ytest,rfpred,average='weighted')) 
    
    clf.append(rf)
    acc.append(m.f1_score(ytest,rfpred,average='weighted'))
    ypredproba_all.append(rf.predict_proba(xtest)) 
    
    confmat=m.confusion_matrix(ytest,rfpred)
    confsumh=np.sum(confmat,axis=0)
    propconfmat=confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[:,i]= 100*propconfmat[:,i]/confsumh[i] 
    ypredconfprob_all.append(propconfmat/100)
    
    
    # =================================================
    # classs 1
    # =================================================
    ytrain = ytrain_original.copy()
    ytest = ytest_original.copy()
    ytrain, ytest = swapcolumns(ytrain, ytest, 1)
    # =================================================
    
    rf = RandomForestClassifier(random_state=randomseed, n_estimators=100)
    rf.fit(xtrain, ytrain)
    rfpred = rf.predict(xtest)
    print(m.f1_score(ytest, rfpred,average='weighted'))
    
    clf.append(rf)
    acc.append(m.f1_score(ytest, rfpred,average='weighted'))
    ypredproba_all.append(rf.predict_proba(xtest))
    
    confmat = m.confusion_matrix(ytest, rfpred)
    confsumh = np.sum(confmat, axis=0)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
    ypredconfprob_all.append(propconfmat / 100) 
            
    #=================================================
    # classs 2
    #=================================================
    
    ytrain=ytrain_original.copy()
    ytest=ytest_original.copy()
    ytrain,ytest= swapcolumns(ytrain,ytest,2)
    
    #=================================================
    
    rf=RandomForestClassifier(random_state=randomseed, n_estimators=100)
    rf.fit(xtrain,ytrain)
    rfpred=rf.predict(xtest)
    print(m.f1_score(ytest,rfpred,average='weighted'))
    
    clf.append(rf)
    acc.append(m.f1_score(ytest,rfpred,average='weighted'))
    ypredproba_all.append(rf.predict_proba(xtest))
    
    confmat=m.confusion_matrix(ytest,rfpred)
    confsumh=np.sum(confmat,axis=0)
    propconfmat=confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[:,i]= 100*propconfmat[:,i]/confsumh[i] 
    ypredconfprob_all.append(propconfmat/100)
    
    # #=================================================
    
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues(acc)
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0]
    finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[1]
    finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[2]
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    # accuracy_score 0.6433333333333333
    
    trial1_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1],
        ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
        ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1]])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0]
    finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[1]
    finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[2]
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    # accuracy_score 0.6783333333333333
    # accuracy_score 0.715
    # accuracy_score 0.7216666666666667
    trial2_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1],
        ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
        ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1]])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = ypredproba_all[0][:, 0] * weightvalga[0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1]
    finalcol[:, 1] = ypredproba_all[1][:, 0] * weightvalga[1] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1]
    finalcol[:, 2] = ypredproba_all[2][:, 0] * weightvalga[2] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1]
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    trial3_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1] ,
        -ypredconfprob_all[0][0][1],
        
        ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
        -ypredconfprob_all[1][0][1],
        
        ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1],
        -ypredconfprob_all[2][0][1],
        
        ])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0] + ypredproba_all[0][:, 1]*weightvalga[1]
    finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[2] + ypredproba_all[1][:, 1]*weightvalga[3]
    finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[4] + ypredproba_all[2][:, 1]*weightvalga[5]
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    
    trial4_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================

    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] ,
        ypredconfprob_all[1][0][0] ,
        ypredconfprob_all[2][0][0] ])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = weightvalga[0] * (ypredproba_all[0][:, 0] * ypredconfprob_all[0][0][0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1])
    finalcol[:, 1] = weightvalga[1] * (ypredproba_all[1][:, 0] * ypredconfprob_all[1][0][0] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1])
    finalcol[:, 2] = weightvalga[2] * (ypredproba_all[2][:, 0] * ypredconfprob_all[2][0][0] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1])
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    trial5_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] ,
        ypredconfprob_all[1][0][0] ,
        ypredconfprob_all[2][0][0] ])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] =  (weightvalga[0] *ypredproba_all[0][:, 0] * ypredconfprob_all[0][0][0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1])
    finalcol[:, 1] = (weightvalga[1] * ypredproba_all[1][:, 0] * ypredconfprob_all[1][0][0] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1])
    finalcol[:, 2] =  (weightvalga[2] *ypredproba_all[2][:, 0] * ypredconfprob_all[2][0][0] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1])
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    trial6_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================

    


KFold(n_splits=10, random_state=42, shuffle=True)
************************** ==>  0
original score 1.0
1.0
1.0
1.0
0
1
2
3
4
5
6
7
8
9
2.884081912371089 [0.92406028 0.96407756 0.99594407]
f1_score 1.0
0
1
2
3
4
5
6
7
8
9
2.901037116499385 [0.9260416  0.98220784 0.99278768]
f1_score 1.0
0
1
2
3
4
5
6
7
8
9
2.896962840012748 [0.92784891 0.97947903 0.9896349 ]
f1_score 1.0
0
1
2
3
4
5
6
7
8
9
2.617706828390788 [0.93486056 0.74133593 0.98193739 0.76753368 0.99104326 0.8944335 ]
f1_score 0.8957264957264957
0
1
2
3
4
5
6
7
8
9
2.858187498975746 [0.8944418  0.96842908 0.99531661]
f1_score 1.0
0
1
2
3
4
5
6
7
8
9
2.9201291933882487 [0.94644056 0.97959407 0.99409456]
f1_score 1.0
************************** ==>  1
original score 1.0
1.0
1.0
1.0
0
1
2
3
4
5
6
7
8
9
2.9042161883214495 [0.9366393  0.9729042  0.99467269]
f1_score 1.0
0
1
2
3
4
5
6
7
8
9
2.930029533542989 [0.94661603 0.98691102 0.99650248]
f1_score 1.0
0
1
2
3
4
5
6
7
8
9
2.901511254487635 [0.913051   0.99369911 0.99476114]
f1_score 

<IPython.core.display.Javascript object>

In [6]:
print("original_scores", np.mean(original_scores), np.std(original_scores))
print("trial1_scores", np.mean(trial1_scores), np.std(trial1_scores))
print("trial2_scores", np.mean(trial2_scores), np.std(trial2_scores))
print("trial3_scores", np.mean(trial3_scores), np.std(trial3_scores))
print("trial2_scores", np.mean(trial4_scores), np.std(trial4_scores))
print("trial3_scores", np.mean(trial5_scores), np.std(trial5_scores))

original_scores 0.9890720390720391 0.02186512652081389
trial1_scores 0.9890720390720391 0.02186512652081389
trial2_scores 0.9890720390720391 0.02186512652081389
trial3_scores 0.9890720390720391 0.02186512652081389
trial2_scores 0.8423154825528407 0.08821780566945177
trial3_scores 0.9943941679235797 0.01681749622926094


<IPython.core.display.Javascript object>